# Feature Engineering

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import cross_validate
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures

In [2]:
df=pd.read_csv('more_data2.csv')
df = df.drop('eng_mfgr', axis = 1)
df = df.drop('acft_make', axis = 1)
df = df.drop('acft_model', axis = 1)
df = df.drop('inj_tot_f', axis = 1)
df = df.drop('inj_tot_s', axis = 1)
df = df.drop('inj_tot_m', axis = 1)
df = df.dropna()

In [33]:
X_number = df.select_dtypes(include='number')
X_category = df.select_dtypes(exclude='number')
X_number = (X_number - X_number.mean())/X_number.std()
X_category = pd.get_dummies(X_category)
X = pd.concat([X_number, X_category], axis =1)
Y=X.inj_tot_t
X = X.drop('inj_tot_t', axis = 1)
Y.head()

27    0.022393
37    0.022393
39    0.470925
40    0.919456
41   -0.426138
Name: inj_tot_t, dtype: float64

In [35]:
#engineer interaction features
poly = PolynomialFeatures(2, interaction_only = True, include_bias = False)
Xe = poly.fit_transform(X)
cols = poly.get_feature_names(X.columns)
Xe = pd.DataFrame(Xe, columns = cols)
#Xe = (Xe - Xe.mean())/Xe.std()
Xe = Xe.dropna(axis=1)
Xe['bias'] = 1
Xe.head(3)

,ev_time,ev_year,sky_nonceil_ht,sky_ceil_ht,vis_sm,wx_temp,wx_dew_pt,wind_vel_kts,gust_kts,dec_latitude,...,eng_type_REC eng_type_TJ,eng_type_REC eng_type_TP,eng_type_REC eng_type_TS,eng_type_TF eng_type_TJ,eng_type_TF eng_type_TP,eng_type_TF eng_type_TS,eng_type_TJ eng_type_TP,eng_type_TJ eng_type_TS,eng_type_TP eng_type_TS,bias
0,0.990709,0.002367,5.663945,-0.223693,0.011537,0.100763,-1.733665,-0.691979,-0.452436,-0.765044,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1,0.334498,0.355313,5.663945,-0.223693,0.011537,-0.481640,-0.696658,0.058737,-0.452436,-0.965213,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
2,1.023601,0.708260,5.663945,-0.223693,0.011537,-0.111020,-0.230004,-0.691979,-0.452436,-1.132837,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


In [36]:
lr = LinearRegression(fit_intercept=False)
lr.fit(Xe, Y)
cv_results = cross_validate(lr, Xe, Y, return_train_score = True)
R2_train = cv_results['train_score'].mean()
R2_valid = cv_results['test_score'].mean()
print('Train R2', R2_train.round(2))
print('Validation R2', R2_valid.round(2))

Train R2 0.86
Validation R2 -2.6500974115357346e+23


In [41]:
coef = pd.Series(lr.coef_, index = Xe.columns)
coef.sort_values().head(25)
#coef = pd.Series(lr.coef_, index = Xe.columns).round(2)
#coef = coef.replace(0, np.nan)
#coef = coef.dropna()
#coef = coef.sort_values(ascending = False)
#coef.plot(x='coeff', figsize=(5, 20), kind="barh")
#coef

sky_nonceil_ht eng_type_TP             -3.289340
acft_year eng_type_REC                 -2.468681
damage_DEST eng_type_TP                -2.211076
rwy_len eng_type_TP                    -1.937841
second_pilot_Y eng_type_TP             -1.920719
afm_hrs_last_insp eng_type_TP          -1.671320
light_cond_NITE eng_type_TP            -1.640992
acft_fire_NONE eng_type_TP             -1.609765
num_eng eng_type_TS                    -1.485557
sky_ceil_ht eng_type_REC               -1.310873
gust_kts eng_type_TF                   -1.213461
wx_temp light_cond_NDRK                -1.176991
ev_time eng_type_REC                   -1.138462
eng_type_TP                            -0.947764
carb_fuel_injection_FINJ eng_type_TP   -0.947764
ev_country_USA eng_type_TP             -0.947764
power_units eng_type_TP                -0.920889
second_pilot_N eng_type_REC            -0.900057
dec_latitude eng_type_TF               -0.838945
wx_temp eng_type_TP                    -0.835104
ev_time eng_type_TS 